In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torchmetrics.classification import MulticlassAccuracy

d:\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train=pd.read_csv('data/train.csv')
df_test=pd.read_csv('data/test.csv')

In [3]:
X=df_train.drop(['Target','id'],axis=1).values
y=df_train['Target'].values

In [4]:
labelencoder=LabelEncoder()
y=labelencoder.fit_transform(y)

In [5]:
scalar=StandardScaler()
X=scalar.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train.shape

(61214, 36)

In [21]:
class ClassificationModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer_1=nn.Sequential(
            nn.Linear(in_features=36,out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64,out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64,out_features=32),
            nn.Tanh(),
            nn.Linear(in_features=32,out_features=3)
        )
    def forward(self,x:torch.Tensor)-> torch.Tensor:
        return self.layer_1(x)

In [8]:
device='cuda'if torch.cuda.is_available() else 'cpu'

In [9]:
model=ClassificationModel()
model.to(device)
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [10]:
X_train = torch.from_numpy(X_train).type(torch.float32).to(device)
X_test = torch.from_numpy(X_test).type(torch.float32).to(device)
y_train = torch.from_numpy(y_train).type(torch.long).to(device)
y_test = torch.from_numpy(y_test).type(torch.long).to(device)

In [11]:
metric = MulticlassAccuracy(num_classes=3).to(device)

In [12]:
def train_model(epochs,model,loss_fn,optimizer,metric,X_train,y_train,X_test,y_test):
    for epoch in range(epochs):
        model.train()

        pred=model(X_train)

        loss=loss_fn(pred,y_train)
        train_acc=metric(pred,y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.inference_mode():
            test_pred=model(X_test)
            test_loss=loss_fn(test_pred,y_test)
            test_acc=metric(test_pred,y_test)
            if epochs % 10 == 0:
                print(f"Epoch: {epoch} | Train loss: {loss} | Train accuracy: {train_acc:.2f}% | Test loss: {test_loss} | Test accuracy: {test_acc:.2f}%")

In [13]:
train_model(
    epochs=100,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    metric=metric,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (61214x36 and 240x120)

In [18]:
test_data=df_test.drop(['id'],axis=1).values
test_data=scalar.fit_transform(test_data)
test_data = torch.from_numpy(test_data).type(torch.float32).to(device)

In [21]:
result=None
model.eval()
with torch.inference_mode():
    result=model(test_data)
result

tensor([[ 0.9534, -0.4670, -0.5872],
        [-0.5539,  0.0780,  1.3358],
        [-0.6680,  0.1121,  1.4137],
        ...,
        [ 1.1256, -0.4432, -0.8247],
        [ 0.2124, -0.1060,  0.3116],
        [ 1.2114, -0.3743, -0.8925]])

In [22]:
result=torch.softmax(result,dim=1)
result

tensor([[0.6869, 0.1660, 0.1472],
        [0.1053, 0.1981, 0.6967],
        [0.0893, 0.1948, 0.7159],
        ...,
        [0.7404, 0.1542, 0.1053],
        [0.3532, 0.2568, 0.3900],
        [0.7537, 0.1544, 0.0919]])

In [23]:
result=result.argmax(dim=1)
result

tensor([0, 2, 2,  ..., 0, 2, 0])

In [24]:
result=result.numpy()
result=labelencoder.inverse_transform(result)
result

array(['Dropout', 'Graduate', 'Graduate', ..., 'Dropout', 'Graduate',
       'Dropout'], dtype=object)

In [ ]:
output = {
    "id": df_test["id"],
    "Target": result
}

output = pd.DataFrame(output)
output.to_csv('submission.csv',index=False)